In [1]:
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import time as t
from astropy import table
from astroquery.heasarc import Heasarc

import pandas as pd

from swifttools.swift_too import ObsQuery, Data
from datetime import date
import os

In [23]:
startpath = os.getcwd()+"/"
initpath = os.getcwd()+"/dataInit"

In [3]:
database=pd.read_csv('psr_list.csv',delimiter=',',header=0)

In [4]:
database

,4FGL,PSR,QUALITY,RA,Dec,Class1
0,4FGL J0002.8+6217,PSR J0002+6216,3,0.7201,62.2905,psr
1,4FGL J0007.0+7303,PSR J0007+7303,3,1.7680,73.0512,psr
2,4FGL J0023.4+0920,PSR J0023+0923,1,5.8513,9.3475,msp
3,4FGL J0030.4+0451,PSR J0030+0451,5,7.6107,4.8592,msp
4,4FGL J0034.3-0534,PSR J0034-0534,3,8.5961,-5.5809,msp
...,...,...,...,...,...,...
315,4FGL J2302.7+4443,PSR J2302+4442,3,345.6918,44.7182,msp
316,4FGL J2310.0-0555,PSR J2310-0555,1,347.5183,-5.9251,msp
317,4FGL J2317.3+1433,PSR J2317+1439,1,349.3349,14.5651,msp
318,4FGL J2333.1-5527,PSR J2333-5526,1,353.2860,-55.4638,msp


In [5]:
tab = Heasarc.query_region(database.iloc[5].to_numpy()[0],radius=15*u.arcmin, catalog="swiftxrlog")
obsinfo=np.asarray(tab)
tab

target_id,obsid,ra,dec,start_time,exposure,window_size,operation_mode,pointing_mode,flipflop,__row
,,deg,deg,d,s,,,,,
int32,object,float64,float64,float64,float64,object,object,object,object,object
31541,00031541001,15.23637,-64.37918,55156.2687449074,12.48625,,BIASIMA,slewing,N,910148
31541,00031541001,15.23637,-64.37918,55156.2692266204,93.77791,200,WINDOWED,slewing,N,910149
31541,00031541001,15.23637,-64.37918,55156.270312037,9.60729,200,WINDOWED,settling,N,910150
31541,00031541001,15.23637,-64.37918,55156.2704486111,149.83724,600x600,PHOTON,pointing,N,910151
31541,00031541001,15.23637,-64.37918,55156.4590225694,12.48625,,BIASIMA,slewing,N,910152
31541,00031541001,15.23637,-64.37918,55156.4595043981,79.90059,200,WINDOWED,slewing,N,910153
31541,00031541001,15.23637,-64.37918,55156.4604378472,9.60731,200,WINDOWED,settling,N,910154
31541,00031541001,15.23637,-64.37918,55156.460562037,521.93312,600x600,PHOTON,pointing,N,910155


In [6]:
df = tab.to_pandas()
df = df.loc[df.pointing_mode == "pointing"].loc[df.operation_mode == "PHOTON"]

In [7]:
df.iloc[1].tolist()

[31541,
 '00031541001',
 15.23637,
 -64.37918,
 55156.460562037,
 521.93312,
 '600x600',
 'PHOTON',
 'pointing',
 'N',
 '910155']

In [8]:
tab[(tab["pointing_mode"]=="pointing") & (tab["operation_mode"]=="PHOTON") & (tab["exposure"] > 100)]

target_id,obsid,ra,dec,start_time,exposure,window_size,operation_mode,pointing_mode,flipflop,__row
,,deg,deg,d,s,,,,,
int32,object,float64,float64,float64,float64,object,object,object,object,object
31541,00031541001,15.23637,-64.37918,55156.2704486111,149.83724,600x600,PHOTON,pointing,N,910151
31541,00031541001,15.23637,-64.37918,55156.460562037,521.93312,600x600,PHOTON,pointing,N,910155
31541,00031541001,15.23637,-64.37918,55156.5272243056,521.93258,600x600,PHOTON,pointing,N,910159
31541,00031541001,15.23637,-64.37918,55156.5938909722,521.93294,600x600,PHOTON,pointing,N,910163
31541,00031541001,15.23637,-64.37918,55156.6612518519,521.93298,600x600,PHOTON,pointing,N,910167
31541,00031541001,15.23637,-64.37918,55156.7313907407,521.93274,600x600,PHOTON,pointing,N,910171
31541,00031541001,15.23637,-64.37918,55156.7980618056,521.93258,600x600,PHOTON,pointing,N,910175


In [18]:
os.chdir(startpath)
duration = []

reset_queries = False

match_list = []
query_list2 = []
done = []

if not reset_queries and os.path.isfile("PSRmatch.csv"):
    if os.path.isfile("temp.csv"):
        os.system("mv temp.csv PSRmatch.csv")
    if os.path.isfile("temp.txt"):
        os.system("mv temp.txt HEASqueries.txt")
    match_df = pd.read_csv("PSRmatch.csv")
    match_list.append("src,ref\n")
    for i in range(len(match_df)):
        ls = match_df.iloc[i].tolist()
        done.append(ls[0])
        string = ""
        for item in ls:
            string += str(item)+","
        string = string[:-1]+"\n"
        match_list.append(string)
    query_list = open("HEASqueries.txt","r")
    # qlines = query_list.readlines()
    query_list2 = query_list.readlines()
    
    # for line in qlines:
    #     if line[-2:] == "\n":
    #         query_list2.append(line[:-2])
    #     else:
    #         print(line)
    #         query_list2.append(line)
    query_list.close()
    os.system("cp PSRmatch.csv temp.csv")
    os.system("cp HEASqueries.txt temp.txt")    

match_file = open("PSRmatch.csv","w")
# query_list = open("HEASqueries.txt","w")
if "src,ref\n" not in match_list:
    match_list.append("src,ref\n")

obs_limit = 10
quality_lim = 3

df = database.loc[database.QUALITY <= quality_lim]

looking_number = len(df)
if looking_number > len(df): looking_number = len(df)

psr_list = df.PSR.to_numpy()[:looking_number]
fgl_list = df["4FGL"].to_numpy()[:looking_number]

print("Looking for",looking_number,"sources")
for name in fgl_list:
    if name in done:
        # print("Skipping "+name)
        continue
    print("Looking for "+name)
    # name = database.iloc[0].to_numpy()[0]

    RA=database.loc[database["4FGL"]==name]['RA'].to_numpy()[0]
    DEC=database.loc[database["4FGL"]==name]['Dec'].to_numpy()[0]
    coords = SkyCoord(RA,DEC, equinox = 'J2000', frame='fk5', unit=(u.deg,u.deg))

    try:
        tab = Heasarc.query_region(coords, catalog="swiftxrlog",radius=15*u.arcmin)
    except:
        tab = Heasarc.query_region(coords, catalog="swiftxrlog",radius=15*u.arcmin)
    if len(tab) == 0: continue
    else:
       tab.sort("exposure",reverse=True)

    # obsinfo=np.asarray(tab[(tab["pointing_mode"]=="pointing") & (tab["operation_mode"]=="PHOTON") & (tab["exposure"] > 100)])
    obsinfo=np.asarray(tab[(tab["pointing_mode"]=="pointing") & (tab["exposure"] > 100)])


    batchquery=[]
    obsdur=0

    n_xrt = 0
    n_uvot = 0

    for j in range(0,len(obsinfo)):    
        if j > 80:
            break
        obsID=str((obsinfo[j][1]))#[2:13]
        # print(obsID)
        time=t.Time(obsinfo[j][4],format='mjd')
        time.format='fits'
        year=(time.value[0:4])
        month=(time.value[5:7])
        #We only want PC mode. Remove this if you want WT mode as well
        if (obsinfo[j][7]==b'PHOTON        '):
            obsdur+=obsinfo[j][5]
        #Need to get observation ID number, stripped of all the useless stuff that piles up.
        # obsID=str((obsinfo[j][1]))[2:13]
        finstring=year+"_"+month+"/"+obsID    
        firststring="wget -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/"
        #Create queries for auxil and xrt datasets.
        query1=firststring+finstring+"/auxil/"
        query2=firststring+finstring+"/xrt/"

        temp = name+','+obsID+'ref\n'
        if temp not in match_list:
            match_list.append(temp)
            n_xrt += 1

        batchquery=np.append(batchquery,query1)
        batchquery=np.append(batchquery,query2) 
        if n_uvot < 5:
            query3=firststring+finstring+"/uvot/"
            # batchquery=np.append(batchquery,query3) 
            # n_uvot += 1
        if n_xrt > obs_limit:
            break

    uniquer=np.unique(batchquery)
    #Write queries to .txt file. The queries can be run however you like.
    for k in range(0,len(uniquer)):
        # query_list.write(uniquer[k]+'\n')
        if uniquer[k] not in query_list2:
            query_list2.append(uniquer[k]+'\n')

    duration=np.append(duration,obsdur)
query_list = open("HEASqueries.txt","w")
for query in query_list2:
    query_list.write(query)
query_list.close()
for line in match_list:
    match_file.write(line)
match_file.close()

if os.path.isfile("temp.csv"):
    os.system("rm temp.*")

print("There are",int(len(match_list)-1),"observations available for the",(looking_number),"sources")

Looking for 256 sources
Looking for 4FGL J0007.0+7303


Looking for 4FGL J0034.3-0534


Looking for 4FGL J0514.6-4408


Looking for 4FGL J0729.3-1445


Looking for 4FGL J0737.9-3041


...

Looking for 4FGL J2052.7+1218


Looking for 4FGL J2124.7-3358


Looking for 4FGL J2205.5+6016


Looking for 4FGL J2208.1+4058


Looking for 4FGL J2310.0-0555
There are 1283 observations available for the 256 sources


In [10]:
tab = Heasarc.query_region("4FGL J0102.8+4839", catalog="swiftxrlog",radius=15*u.arcmin)


In [11]:
tab.sort("exposure",reverse=True)

In [12]:
tab[(tab["pointing_mode"]=="pointing")&(tab["exposure"]>100)]

target_id,obsid,ra,dec,start_time,exposure,window_size,operation_mode,pointing_mode,flipflop,__row
,,deg,deg,d,s,,,,,
int32,object,float64,float64,float64,float64,object,object,object,object,object
41270,00041270004,15.74953,48.65474,55503.5784449074,894.02633,600x600,PHOTON,pointing,N,264150
41270,00041270004,15.74953,48.65474,55503.254034375,724.21037,600x600,PHOTON,pointing,N,264138
41270,00041270004,15.74953,48.65474,55503.511815162,711.72479,600x600,PHOTON,pointing,N,264144
41270,00041270001,15.79604,48.63667,55481.5716165509,526.92671,600x600,PHOTON,pointing,N,263887
41270,00041270001,15.79604,48.63667,55481.50505625,516.93734,600x600,PHOTON,pointing,N,263882
41270,00041270003,15.80840,48.62944,55501.3282974537,249.72753,600x600,PHOTON,pointing,N,263820
41270,00041270003,15.80840,48.62944,55501.3033300926,247.23164,600x600,PHOTON,pointing,N,263814
41270,00041270002,15.73669,48.65139,55492.5451076389,237.24207,600x600,PHOTON,pointing,N,264087


In [13]:
obsinfo=np.asarray(tab[(tab["pointing_mode"]=="pointing") & (tab["exposure"] > 100)])


In [14]:
raise ValueError("STOP")

ValueError: STOP

In [25]:
os.chdir(initpath)
counts = 20
x_ls = list(np.linspace(1/counts,1,counts))

openfile = open(startpath+"HEASqueries.txt","r")
current_query = openfile.readlines()
openfile.close()

# current_query = query_list2

print("0% Downloaded")

for i,query in enumerate(current_query):
    dat = query.split("/")
    loc = dat[-3]+"/"+dat[-2]
    # print(dat)
    # print(loc)

    for x in x_ls:
        # print(i%int(x*len(current_query)))
        # print(i%int(x*len(query_list2)) == 0)
        if (1+i)%int(x*len(current_query)) == 0:
            print(str(int(x*100))+"% Downloaded")
            x_ls = x_ls[1:]

    if os.path.isdir(loc):
        continue
    # print(query)
    # print("\n")
    
            
    os.system(query)

5% Downloaded
10% Downloaded
15% Downloaded
20% Downloaded
25% Downloaded
30% Downloaded
35% Downloaded
40% Downloaded
44% Downloaded
49% Downloaded
54% Downloaded
60% Downloaded
65% Downloaded
70% Downloaded
75% Downloaded
80% Downloaded
85% Downloaded
90% Downloaded
95% Downloaded
100% Downloaded


## swifttools test

In [ ]:
target = "1ES 1028+511"
# loc = os.getcwd()+"/"+"V1405_new/dataInit/"
loc = os.getcwd()+"/"+target.replace(" ","_")+"/dataInit/"

begin = date(2015,1,1)#date(2024,10,23)


In [ ]:
# # psr_list = database.PSR.to_numpy()[:1]
# overwrite = False

# os.chdir(initpath)

# for name in psr_list:
#     query = ObsQuery(name=name)

#     n = 1

#     for ind, q in enumerate(query):
#         if q.exposure.seconds >= 100:
#             if (os.path.isdir(str(query[ind].obsid)) and not overwrite) or n > 3:
#                 # print("Skipping",q.obsid)
#                 pass
#             else:
#                 n += 1
#                 print("Downloading",query[ind].obsid)
#                 Data(obsid=query[ind].obsid, xrt=True, clobber=overwrite)

In [ ]:
query= ObsQuery(name="V1405 Cas")